In [4]:
import pandas as pd

In [5]:
#Global paramenters
#API URLs from RxNorm
base_endpoint = "https://api.fda.gov/drug/ndc.json?search=active_ingredients.name:"
limit = "&limit=1000"
#dataframe
df = pd.DataFrame()

In [6]:
#Method to get all drugs with an opioid ingredient
def query_drug(name_of_drug):
    row = 0
    df = pd.DataFrame()
    df = pd.read_json(base_endpoint+name_of_drug+limit, orient='values', typ='series', convert_dates=False)
    new_df = pd.DataFrame(df.results)
    return new_df

In [6]:
ingredients = ["alfentanil","benzhydrocodone","buprenorphine","butorphanol","codeine","dextromethorphan","difenoxin","dihydrocodeine","diphenoxylate","fentanyl","hydrocodone","hydromorphone","levorphanol","methadone","meperidine","morphine","nalbuphine","noscapine","oliceridine","opium","oxycodone","oxymorphone","remifentanil","sufentanil","tapentadol","tetrapeptide","tramadol"]

In [7]:
df_opioid = pd.DataFrame()
for drug in ingredients:
    print(drug)
    merg_df = query_drug(drug)
    df_opioid = pd.concat([df_opioid, merg_df]).drop_duplicates(subset=['product_ndc']).reset_index(drop=True)
df_opioid

alfentanil
benzhydrocodone
buprenorphine
butorphanol
codeine
dextromethorphan
difenoxin
dihydrocodeine
diphenoxylate
fentanyl
hydrocodone
hydromorphone
levorphanol
methadone
meperidine
morphine
nalbuphine
noscapine
oliceridine
opium
oxycodone
oxymorphone
remifentanil
sufentanil
tapentadol
tetrapeptide
tramadol


,product_ndc,generic_name,labeler_name,dea_schedule,brand_name,active_ingredients,finished,packaging,listing_expiration_date,openfda,...,spl_id,product_type,route,marketing_start_date,product_id,application_number,brand_name_base,pharm_class,marketing_end_date,brand_name_suffix
0,17478-067,Alfentanil Hydrochloride,Akorn,CII,Alfentanil Hydrochloride,"[{'name': 'ALFENTANIL HYDROCHLORIDE', 'strengt...",True,"[{'package_ndc': '17478-067-02', 'description'...",20231231,"{'manufacturer_name': ['Akorn'], 'rxcui': ['17...",...,aea7d858-c41e-4726-8678-9c2bf08e5d56,HUMAN PRESCRIPTION DRUG,[INTRAVENOUS],20100201,17478-067_aea7d858-c41e-4726-8678-9c2bf08e5d56,NDA019353,Alfentanil Hydrochloride,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
1,49812-0033,Alfentanil HCl,Johnson Matthey Inc.,CII,NaN,"[{'name': 'ALFENTANIL HYDROCHLORIDE', 'strengt...",False,"[{'package_ndc': '49812-0033-1', 'description'...",20221231,{},...,97f61de8-b632-c2a9-e053-2995a90ab7d5,BULK INGREDIENT,NaN,19970919,49812-0033_97f61de8-b632-c2a9-e053-2995a90ab7d5,NaN,None,NaN,NaN,NaN
2,49812-0220,Hydrocodone Bitartrate Enol Benzoate HCl,Johnson Matthey Inc.,NaN,NaN,"[{'name': 'BENZHYDROCODONE HYDROCHLORIDE', 'st...",False,"[{'package_ndc': '49812-0220-1', 'description'...",20221231,{},...,bea0a30e-6b67-4188-b3b6-f32aa3d56cb0,BULK INGREDIENT,NaN,20140101,49812-0220_bea0a30e-6b67-4188-b3b6-f32aa3d56cb0,NaN,None,NaN,NaN,NaN
3,10702-342,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",CII,APADAZ,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '10702-342-01', 'description'...",20221231,"{'manufacturer_name': ['KVK-Tech, Inc.'], 'rxc...",...,bd5dd034-4b41-2f37-e053-2995a90a4f77,HUMAN PRESCRIPTION DRUG,[ORAL],20190628,10702-342_bd5dd034-4b41-2f37-e053-2995a90a4f77,NDA208653,APADAZ,NaN,NaN,NaN
4,10702-340,Benzhydrocodone and Acetaminophen,"KVK-Tech, Inc.",CII,APADAZ,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '10702-340-01', 'description'...",20221231,"{'manufacturer_name': ['KVK-Tech, Inc.'], 'rxc...",...,bd5dd034-4b41-2f37-e053-2995a90a4f77,HUMAN PRESCRIPTION DRUG,[ORAL],20190628,10702-340_bd5dd034-4b41-2f37-e053-2995a90a4f77,NDA208653,APADAZ,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3046,47335-861,TRAMADOL HYDROCHLORIDE,"Sun Pharmaceutical Industries, Inc.",CIV,TRAMADOL HYDROCHLORIDE,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '47335-861-08', 'description'...",20221231,{'manufacturer_name': ['Sun Pharmaceutical Ind...,...,cffa4953-1dcd-4abd-a589-68d667292ed2,HUMAN PRESCRIPTION DRUG,[ORAL],20111212,47335-861_cffa4953-1dcd-4abd-a589-68d667292ed2,ANDA201384,TRAMADOL HYDROCHLORIDE,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
3047,69543-136,TRAMADOL HYDROCHLORIDE,Virtus Pharmaceuticals LLC,CIV,TRAMADOL HYDROCHLORIDE,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '69543-136-10', 'description'...",NaN,{'manufacturer_name': ['Virtus Pharmaceuticals...,...,1065b54d-328a-491b-aa20-eaddebd0c119,HUMAN PRESCRIPTION DRUG,[ORAL],20150522,69543-136_1065b54d-328a-491b-aa20-eaddebd0c119,ANDA091498,TRAMADOL HYDROCHLORIDE,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",20230621,NaN
3048,61919-073,TRAMADOL HYDROCHLORIDE,DIRECT RX,CIV,TRAMADOL HYDROCHLORIDE,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",True,"[{'package_ndc': '61919-073-21', 'description'...",20231231,"{'manufacturer_name': ['DIRECT RX'], 'rxcui': ...",...,d554a939-5fac-2c91-e053-2a95a90a36c2,HUMAN PRESCRIPTION DRUG,[ORAL],20180615,61919-073_d554a939-5fac-2c91-e053-2a95a90a36c2,ANDA076003,TRAMADOL HYDROCHLORIDE,"[Full Opioid Agonists [MoA], Opioid Agonist [E...",NaN,NaN
3049,46438-1108,Tramadol Hydrochloride,Dipharma Francis Srl,CIV,NaN,"[{'name': 'TRAMADOL HYDROCHLORIDE', 'strength'...",False,"[{'package_ndc': '46438-1108-1', 'description'...",20221231,{},...,d343306b-2d95-6aff-e053-2995a90a7693,BULK INGREDIENT,NaN,2012

In [20]:
query_drug("hydrocodone")

,product_ndc,generic_name,labeler_name,dea_schedule,brand_name,active_ingredients,finished,packaging,listing_expiration_date,openfda,...,spl_id,product_type,route,marketing_start_date,product_id,application_number,brand_name_base,pharm_class,marketing_end_date,brand_name_suffix
0,50090-1517,Hydrocodone Bitartrate And Acetaminophen,A-S Medication Solutions,CII,Hydrocodone Bitartrate And Acetaminophen,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '50090-1517-0', 'description'...",20221231,{'manufacturer_name': ['A-S Medication Solutio...,...,6f581671-9584-4739-9369-5a215e5b6ed0,HUMAN PRESCRIPTION DRUG,[ORAL],20060119,50090-1517_6f581671-9584-4739-9369-5a215e5b6ed0,ANDA040655,Hydrocodone Bitartrate And Acetaminophen,"[Opioid Agonist [EPC], Opioid Agonists [MoA]]",NaN,NaN
1,63629-5432,Hydrocodone Bitartrate And Acetaminophen,Bryant Ranch Prepack,CII,Hydrocodone Bitartrate And Acetaminophen,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '63629-5432-0', 'description'...",NaN,{'manufacturer_name': ['Bryant Ranch Prepack']...,...,00f1e7e6-4f94-4bf4-a726-0b4ffbf6d1b1,HUMAN PRESCRIPTION DRUG,[ORAL],20000531,63629-5432_00f1e7e6-4f94-4bf4-a726-0b4ffbf6d1b1,ANDA040355,Hydrocodone Bitartrate And Acetaminophen,"[Opioid Agonist [EPC], Opioid Agonists [MoA]]",20230930,NaN
2,0406-0125,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,SpecGx LLC,CII,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '0406-0125-01', 'description'...",20221231,"{'manufacturer_name': ['SpecGx LLC'], 'rxcui':...",...,d8235792-ff67-48ec-b06f-0a6fafdd15dd,HUMAN PRESCRIPTION DRUG,[ORAL],20000726,0406-0125_d8235792-ff67-48ec-b06f-0a6fafdd15dd,ANDA040400,HYDROCODONE BITARTRATE AND ACETAMINOPHEN,"[Opioid Agonist [EPC], Opioid Agonists [MoA]]",NaN,NaN
3,40032-810,Hydrocodone Bitartrate and Homatropine Methylb...,"Novel Laboratories, Inc.",CII,Hydrocodone Bitartrate and Homatropine Methylb...,"[{'name': 'HOMATROPINE METHYLBROMIDE', 'streng...",True,"[{'package_ndc': '40032-810-05', 'description'...",20221231,"{'manufacturer_name': ['Novel Laboratories, In...",...,c3798312-734f-4a19-b9fd-be05faee5ff1,HUMAN PRESCRIPTION DRUG,[ORAL],20170213,40032-810_c3798312-734f-4a19-b9fd-be05faee5ff1,ANDA203535,Hydrocodone Bitartrate and Homatropine Methylb...,"[Opioid Agonist [EPC], Opioid Agonists [MoA]]",NaN,NaN
4,17856-0455,hydrocodone bitartrate and homatropine methylb...,ATLANTIC BIOLOGICALS CORP.,CII,Hydrocodone Bitartrate and Homatropine Methylb...,"[{'name': 'HOMATROPINE METHYLBROMIDE', 'streng...",True,"[{'package_ndc': '17856-0455-1', 'description'...",20221231,{'manufacturer_name': ['ATLANTIC BIOLOGICALS C...,...,4c810449-5c1a-4d86-86ee-7a6cb59b02d7,HUMAN PRESCRIPTION DRUG,[ORAL],19830303,17856-0455_4c810449-5c1a-4d86-86ee-7a6cb59b02d7,ANDA088008,Hydrocodone Bitartrate and Homatropine Methylb...,"[Opioid Agonist [EPC], Opioid Agonists [MoA]]",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
320,63629-1938,Hydrocodone Bitartrate and Acetaminophen,Bryant Ranch Prepack,CII,Hydrocodone Bitartrate and Acetaminophen,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '63629-1938-1', 'description'...",20231231,{'manufacturer_name': ['Bryant Ranch Prepack']...,...,cd6cfd67-3a27-44e7-92d3-00be7946ca74,HUMAN PRESCRIPTION DRUG,[ORAL],20180713,63629-1938_cd6cfd67-3a27-44e7-92d3-00be7946ca74,ANDA210211,Hydrocodone Bitartrate and Acetaminophen,"[Opioid Agonist [EPC], Opioid Agonists [MoA]]",NaN,NaN
321,71930-019,Hydrocodone Bitartrate and Acetaminophen,Eywa Pharma Inc,CII,Hydrocodone Bitartrate and Acetaminophen,"[{'name': 'ACETAMINOPHEN', 'strength': '325 mg...",True,"[{'package_ndc': '71930-019-12', 'description'...",20221231,"{'manufacturer_name': ['Eywa Pharma Inc'], 'rx...",...,58b53a57-388e-40d0-9985-048e5af09b0d,HUMAN PRESCRIPTION DRUG,[ORAL],20180713,71930-019_58b53a57-388e-40d0-9985-048e5af09b0d,ANDA210

In [18]:
df_opioid = pd.DataFrame(query_drug("hydrocodone"))

In [19]:
#get within package
df_package = pd.DataFrame()
i = 0
while i < df_opioid.shape[0]:
    df_temp = pd.DataFrame(df_opioid.packaging[i])
    df_package = pd.concat([df_package, df_temp]).drop_duplicates(subset=['package_ndc']).reset_index(drop=True)
    i+=1
df_package

,package_ndc,description,marketing_start_date,sample,marketing_end_date
0,50090-1517-0,"20 TABLET in 1 BOTTLE, PLASTIC (50090-1517-0)",20141211,False,NaN
1,50090-1517-7,"12 TABLET in 1 BOTTLE, PLASTIC (50090-1517-7)",20151204,False,NaN
2,63629-5432-0,180 TABLET in 1 BOTTLE (63629-5432-0),20150423,False,20230930
3,63629-5432-1,10 TABLET in 1 BOTTLE (63629-5432-1),20141201,False,20230930
4,63629-5432-2,12 TABLET in 1 BOTTLE (63629-5432-2),20160523,False,20230930
...,...,...,...,...,...
919,71335-1248-9,120 TABLET in 1 BOTTLE (71335-1248-9),20211228,False,NaN
920,71930-020-12,100 TABLET in 1 BOTTLE (71930-020-12),20180713,False,NaN
921,71930-020-52,500 TABLET in 1 BOTTLE (71930-020-52),20180713,False,NaN
922,71930-021-12,100 TABLET in 1 BOTTLE (71930-021-12),20180713,False,NaN


In [21]:
#Change to NDC11
for ele in df_package.package_ndc:
    s = ele.split("-")
    S = []
    for i, L in enumerate([5,4,2]):
        if len(s[i]) < L:
            S.append('0'*(L-len(s[i])) + s[i])
        else:
            S.append(s[i])
    ndc11 = ''.join(S)
    df_package.replace(ele,ndc11, inplace=True)
df_package

,package_ndc,description,marketing_start_date,sample,marketing_end_date
0,50090151700,"20 TABLET in 1 BOTTLE, PLASTIC (50090-1517-0)",20141211,False,NaN
1,50090151707,"12 TABLET in 1 BOTTLE, PLASTIC (50090-1517-7)",20151204,False,NaN
2,63629543200,180 TABLET in 1 BOTTLE (63629-5432-0),20150423,False,20230930
3,63629543201,10 TABLET in 1 BOTTLE (63629-5432-1),20141201,False,20230930
4,63629543202,12 TABLET in 1 BOTTLE (63629-5432-2),20160523,False,20230930
...,...,...,...,...,...
919,71335124809,120 TABLET in 1 BOTTLE (71335-1248-9),20211228,False,NaN
920,71930002012,100 TABLET in 1 BOTTLE (71930-020-12),20180713,False,NaN
921,71930002052,500 TABLET in 1 BOTTLE (71930-020-52),20180713,False,NaN
922,71930002112,100 TABLET in 1 BOTTLE (71930-021-12),20180713,False,NaN


In [22]:
df_package.to_excel("ndc_opioidsHYDRO.xlsx", index=False)

In [12]:
df_cdc = pd.read_excel("CDC_opioid NDC_oral MME conversion_update_2020.xlsx", sheet_name="Opioids", dtype='object')
df_cdc

,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
15294,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
15295,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
15296,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
15297,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [13]:
#Identify new drugs found
same_drug_count = 0
new_drug_count = 0
drug_not_found_count = 0
same_drug = pd.DataFrame()
new_drug = pd.DataFrame()
for code in df_package['package_ndc']:
    if df_cdc.isin([str(code)]).any().any() == True:
        same_drug_count += 1
        same_drug = pd.concat([same_drug, df_package.loc[df_package['package_ndc']==code]]).reset_index(drop=True)
    else:
        new_drug_count += 1
        new_drug = pd.concat([new_drug, df_package.loc[df_package['package_ndc']==code]]).reset_index(drop=True)
print('Same drug count: ', same_drug_count)
print('New drug count: ', new_drug_count)

Same drug count:  1896
New drug count:  4327


In [15]:
absent_drugs = pd.DataFrame()
count = 0
#for every code from the combind excel list
for code in df_cdc['NDC']:
    if df_package.isin([str(code)]).any().any() == True:
        count+=1
    else:
        absent_drugs = pd.concat([absent_drugs, df_cdc.loc[df_cdc['NDC']==code]]).reset_index(drop=True)
print('Count: ', count)
absent_drugs

Count:  1896


,NDC,NDC_Numeric,PRODNME,GENNME,Master_Form,Class,Drug,LongShortActing,DEAClassCode,Strength_Per_Unit,UOM,MME_Conversion_Factor
0,00002035102,2035102,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
1,00002035103,2035103,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
2,00002035133,2035133,DARVOCET-N 50,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23
3,00002035302,2035302,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
4,00002035303,2035303,DARVON-N,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
...,...,...,...,...,...,...,...,...,...,...,...,...
13398,99999700061,99999700061,PROPOXYPHENE NAPSYLATE,Propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,100,MG,0.23
13399,99999875926,99999875926,APAP W/HYDROCODONE,Acetaminophen/hydrocodone Bitartrate,Capsule,Opioid,Hydrocodone SA,SA,2,5,MG,1
13400,99999900000,99999900000,ACETAMINOPHEN W/CODEINE,Acetaminophen/codeine Phosphate,Tablet,Opioid,Codeine,SA,3,30,MG,0.15
13401,99999987600,99999987600,APAP W/PROPOXYPHENE NAPSYLATE,Acetaminophen/propoxyphene Napsylate,Tablet,Opioid,Propoxyphene,SA,4,50,MG,0.23


In [16]:
#export
same_drug.to_excel("Same_Opioid_Medications.xlsx", index=False)
new_drug.to_excel("New_Opioids_Added.xlsx", index=False)
absent_drugs.to_excel("Absent_Opioid.xlsx", index=False)